In [1]:
#BLOQUE DE IMPORTAR LIBRERIAS

import requests
import time
import ast
import pandas as pd  # Agregar esta línea

In [ ]:
#BLOQUE DE METODO PARA OBTENER GENEROS Y CATEGORIAS

import requests
import time
import pandas as pd  # Agregar esta línea

# Método para obtener la información del juego
def obtener_tags_juego(app_id):
    url = f"http://store.steampowered.com/api/appdetails?appids={app_id}"
    try:
        respuesta = requests.get(url, timeout=10)  # Timeout de 10 segundos
        respuesta.raise_for_status()  # Lanzar una excepción para errores HTTP
        detalles_juego = respuesta.json().get(str(app_id), {})
        if 'success' in detalles_juego and detalles_juego['success'] is False:
            print(f"No se pudo obtener información para el juego con ID {app_id}.")
            return [], []
        if 'data' in detalles_juego:
            generos = [genero['description'] for genero in detalles_juego['data'].get('genres', [])]
            categorias = [categoria['description'] for categoria in detalles_juego['data'].get('categories', [])]
            return generos, categorias
    except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud para el juego con ID {app_id}: {e}")

    return [], []

# Leer el archivo CSV completo
df_csv = pd.read_csv("C:/Users/oem/Videos/Steam Big Data 110324/proyecto-big-data/data/ID Juegos Division/Adri.csv")

# Crear columnas para Géneros y Categorías
df_csv['Géneros'] = ""
df_csv['Categorías'] = ""

# Contador para el número de ejecuciones
num_ejecuciones = 0

# Iterar sobre la columna "ID del juego" y obtener las tags para cada juego
for index, row in df_csv.iterrows():
    num_ejecuciones += 1
    app_id = row['ID del juego']
    generos, categorias = obtener_tags_juego(app_id)
    if not generos and not categorias:
        print(f"No se obtuvieron tags para el juego con ID {app_id}.")
    else:
        df_csv.at[index, 'Géneros'] = ', '.join(generos)
        df_csv.at[index, 'Categorías'] = ', '.join(categorias)
        # Guardar el DataFrame actualizado con las nuevas columnas después de cada iteración
        df_csv.to_csv(f"C:/Users/oem/Videos/Steam Big Data 110324/proyecto-big-data/data/ID Juegos Division/AdriFull.csv", index=False)
    time.sleep(1.14)  # Esperar 1 segundo entre cada solicitud

print(f"Se han procesado {num_ejecuciones} juegos.")

In [ ]:
#BLOQUE METODO PARA OBTENER FECHA DE LANZAMIENTO

import pandas as pd
import requests

# Método para obtener la fecha de lanzamiento del juego
def obtener_fecha_lanzamiento(app_id):
    url = f"http://store.steampowered.com/api/appdetails?appids={app_id}"
    try:
        respuesta = requests.get(url, timeout=10)
        respuesta.raise_for_status()
        detalles_juego = respuesta.json().get(str(app_id), {})
        if 'data' in detalles_juego:
            fecha_lanzamiento = detalles_juego['data'].get('release_date', None)
            return fecha_lanzamiento
    except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud para el juego con ID {app_id}: {e}")
    return None

# Leer el archivo CSV con el DataFrame
df = pd.read_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/AdriFull.csv")

# Aplicar la función para obtener la fecha de lanzamiento a cada fila del DataFrame
df['Fecha de Lanzamiento'] = df['ID del juego'].apply(obtener_fecha_lanzamiento)

# Guardar el DataFrame con la nueva columna
df.to_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/gameTags_fechafull_adri.csv", index=False)


In [4]:
#BLOQUE METODO PARA OBTENER MULTIPLAYER SI Ó NO (1 Ó 0)

import pandas as pd
import numpy as np  # Importa numpy para manejar los valores nulos

# Leer el archivo CSV con el DataFrame
df = pd.read_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/gameTags_fechafull_adri.csv")

# Definir una función para asignar valores a la columna "Multiplayer"
def asignar_valor_multiplayer(categorias):
    if isinstance(categorias, str):  # Verifica si la categoría es una cadena
        if "Multi-player" in categorias:
            return 1
        else:
            return 0
    else:
        return np.nan  # Devuelve NaN para valores no válidos

# Aplicar la función a la columna "Categorías" para crear la columna "Multiplayer"
df['Multiplayer'] = df['Categorías'].apply(asignar_valor_multiplayer)

# Guardar el DataFrame con la nueva columna en un nuevo archivo CSV
df.to_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/dataframe_date_multiplayer_adri.csv", index=False)

In [11]:
df.dtypes

Unnamed: 0                int64
ID del juego              int64
Géneros                  object
Categorías               object
Fecha de Lanzamiento     object
Fecha                    object
Dias_Pasados              int64
Multiplayer             float64
dtype: object

In [5]:
#BLOQUE METODO PARA OBTENER CANTIDAD DE DESCARGAS

import pandas as pd
import requests

# Función para obtener la cantidad de descargas de un juego por su ID
def obtener_cantidad_descargas(app_id):
    url = f"https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid={app_id}"
    try:
        respuesta = requests.get(url)
        respuesta.raise_for_status()
        data = respuesta.json()
        if 'response' in data and 'player_count' in data['response']:
            return data['response']['player_count']
        else:
            return "Cantidad de descargas no disponible"
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener la cantidad de descargas para el juego con ID {app_id}: {e}")
        return "Error"

# DataFrame con los IDs de los juegos
df = pd.read_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/dataframe_date_multiplayer_adri.csv")

# Aplicar la función a cada ID del juego en el DataFrame y agregar los resultados como una nueva columna
df['Cantidad de descargas'] = df['ID del juego'].apply(obtener_cantidad_descargas)

# Imprimir el DataFrame resultante
print(df)
# Guardar el DataFrame con la nueva columna en un nuevo archivo CSV
df.to_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/dataframe_descargas_adri.csv", index=False)


Error al obtener la cantidad de descargas para el juego con ID 952440: 404 Client Error: Not Found for url: https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid=952440
Error al obtener la cantidad de descargas para el juego con ID 468630: 404 Client Error: Not Found for url: https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid=468630
Error al obtener la cantidad de descargas para el juego con ID 840820: 404 Client Error: Not Found for url: https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid=840820
Error al obtener la cantidad de descargas para el juego con ID 306910: 404 Client Error: Not Found for url: https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid=306910
Error al obtener la cantidad de descargas para el juego con ID 1714900: 404 Client Error: Not Found for url: https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid=1714900
Error al obtener l

In [6]:
#BLOQUE METODO PARA OBTENER PRECIO

import pandas as pd
import requests

# Función para obtener el precio de un juego por su ID utilizando SteamSpy
def obtener_precio_juego(app_id):
    url = f"https://steamspy.com/api.php?request=appdetails&appid={app_id}"
    try:
        respuesta = requests.get(url)
        respuesta.raise_for_status()
        data = respuesta.json()
        if 'price' in data:
            return data['price']
        else:
            return "Precio no disponible"
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener el precio para el juego con ID {app_id}: {e}")
        return "Error"

# DataFrame con los IDs de los juegos
df = pd.read_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/dataframe_descargas_adri.csv")

# Aplicar la función a cada ID del juego en el DataFrame y agregar los resultados como una nueva columna
df['Precio'] = df['ID del juego'].apply(obtener_precio_juego)

# Imprimir el DataFrame resultante
print(df)
# Guardar el DataFrame con la nueva columna en un nuevo archivo CSV
df.to_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/dataframe_precio_adri.csv", index=False)


Error al obtener el precio para el juego con ID 333860: Expecting value: line 1 column 1 (char 0)
Error al obtener el precio para el juego con ID 335920: Expecting value: line 1 column 1 (char 0)
Error al obtener el precio para el juego con ID 395480: Expecting value: line 1 column 1 (char 0)
Error al obtener el precio para el juego con ID 569430: Expecting value: line 1 column 1 (char 0)
Error al obtener el precio para el juego con ID 684580: Expecting value: line 1 column 1 (char 0)
Error al obtener el precio para el juego con ID 958530: Expecting value: line 1 column 1 (char 0)
Error al obtener el precio para el juego con ID 978300: Expecting value: line 1 column 1 (char 0)
Error al obtener el precio para el juego con ID 677480: Expecting value: line 1 column 1 (char 0)
Error al obtener el precio para el juego con ID 668610: Expecting value: line 1 column 1 (char 0)
Error al obtener el precio para el juego con ID 568830: Expecting value: line 1 column 1 (char 0)
Error al obtener el 

In [8]:
#BLOQUE METODO PARA OBTENER SI HAY Ó NO MICROTRANSACCIONES DENTRO DEL JUEGO (1 Ó 0)

import pandas as pd

# Leer el archivo CSV con el DataFrame
df = pd.read_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/dataframe_precio_adri.csv")

# Función para asignar valores a la columna "In-Purchased"
def asignar_valor_in_purchased(categorias):
    if isinstance(categorias, str):  # Verifica si la categoría es una cadena
        if "In-App Purchases" in categorias:
            return 1
        else:
            return 0
    else:
        return 0  # Devuelve 0 para valores no válidos o NaN

# Aplicar la función a la columna "Categorías" para crear la columna "In-Purchased"
df['In-Purchased'] = df['Categorías'].apply(asignar_valor_in_purchased)

# Guardar el DataFrame actualizado en un nuevo archivo CSV
df.to_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/new_dataframe_full_adri.csv", index=False)

# Imprimir el DataFrame resultante
print(df)


      ID del juego                      Géneros  \
0           330450        Action, Casual, Indie   
1           330680                Action, Indie   
2           275060            Action, Adventure   
3           331190                Action, Indie   
4           331220                Action, Indie   
...            ...                          ...   
4551        387400      Action, Indie, Strategy   
4552        400030  Casual, Free to Play, Indie   
4553        882110      Adventure, Free to Play   
4554       1265860               Racing, Sports   
4555       1089830   Racing, Simulation, Sports   

                                             Categorías  \
0     Single-player, Multi-player, Steam Achievement...   
1     Single-player, Steam Achievements, Steam Tradi...   
2     Single-player, Multi-player, Co-op, Steam Achi...   
3     Single-player, Full controller support, Steam ...   
4     Multi-player, Shared/Split Screen, Steam Achie...   
...                              

In [22]:
df['Precio'] = df['Precio'].fillna(0)

In [23]:
df

,Unnamed: 0,ID del juego,Géneros,Categorías,Fecha de Lanzamiento,Fecha,Dias_Pasados,Multiplayer,Cantidad de descargas,Precio,In-Purchased
0,0,461640,"Action, Adventure, Indie, RPG","Single-player, Steam Achievements, Full contro...",NaN,-1,2457,0.0,1,55.0,0
1,1,730,"Action, Free to Play","Multi-player, Cross-Platform Multiplayer, Stea...",NaN,-1,2457,1.0,708661,0.0,1
2,2,578080,"Action, Adventure, Massively Multiplayer, Free...","Multi-player, PvP, Online PvP, Stats, Remote P...",NaN,-1,2457,1.0,166412,0.0,0
3,3,570,"Action, Strategy, Free to Play","Multi-player, Co-op, Steam Trading Cards, Stea...",NaN,-1,2457,1.0,272098,0.0,1
4,4,42690,NaN,NaN,NaN,-1,2457,NaN,49,3999.0,0
...,...,...,...,...,...,...,...,...,...,...,...
4549,4549,777130,"Casual, Indie, Simulation, Free to Play","Single-player, Steam Achievements, In-App Purc...","{'coming_soon': False, 'date': '22 Apr, 2020'}",2020-04-22 00:00:00,1442,0.0,503,0.0,1
4550,4550,784950,"Free to Play, Indie, Massively Multiplayer, St...","Multi-player, MMO, PvP, Online PvP, Co-op, Onl...","{'coming_soon': False, 'date': '15 Mar, 2018'}",2018-03-15 00:00:00,2211,1.0,409,0.0,1
4551,4551,790600,"Free to Play, Indie, Strategy","Multi-player, PvP, Online PvP, Steam Achieveme...","{'coming_soon': False, 'date': '15 May, 2018'}",2018-05-15 00:00:00,2150,1.0,15,0.0,1
4552,4552,793350,"Action, Casual, Indie","Single-player, Steam Achievements, Full contro...","{'coming_soon': False, 'date': '12 Mar, 2018'}",2018-03-12 00:00:00,2214,0.0,1,299.0,0


In [3]:
df = pd.read_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/new_dataframe_full_roy.csv")
df.dtypes

ID del juego               int64
Géneros                   object
Categorías                object
Fecha de Lanzamiento      object
Multiplayer              float64
Cantidad de descargas     object
Precio                   float64
In-Purchased               int64
dtype: object

In [9]:
#BLOQUE METODO PARA ELIMINAR EL "COMING SOON" DE LOS REGISTROS DE LA FECHA DE LANZAMIENTO

import pandas as pd
import numpy as np
import ast

# DataFrame de ejemplo con una columna "Fecha de Lanzamiento" en formato de cadena
df = pd.read_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/new_dataframe_full_adri.csv")

# Función para limpiar los registros y conservar solo la parte de la fecha
def limpiar_fecha(registro):
    if isinstance(registro, str):  # Verificar si el registro es una cadena
        # Convertir el registro de cadena a un diccionario usando ast.literal_eval()
        registro_dict = ast.literal_eval(registro)
        # Crear un nuevo diccionario solo con la parte 'date'
        nuevo_registro_dict = {'date': registro_dict['date']}
        # Convertir el diccionario de nuevo a cadena
        nuevo_registro = str(nuevo_registro_dict)
        return nuevo_registro
    else:
        return np.nan  # Devolver NaN para valores nulos

# Aplicar la función a la columna "Fecha de Lanzamiento" para limpiar los registros
df['Fecha de Lanzamiento'] = df['Fecha de Lanzamiento'].apply(limpiar_fecha)

# Imprimir el DataFrame resultante
print(df)
df.to_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/cambio_fechas_full_adri.csv", index=False)


      ID del juego                      Géneros  \
0           330450        Action, Casual, Indie   
1           330680                Action, Indie   
2           275060            Action, Adventure   
3           331190                Action, Indie   
4           331220                Action, Indie   
...            ...                          ...   
4551        387400      Action, Indie, Strategy   
4552        400030  Casual, Free to Play, Indie   
4553        882110      Adventure, Free to Play   
4554       1265860               Racing, Sports   
4555       1089830   Racing, Simulation, Sports   

                                             Categorías  \
0     Single-player, Multi-player, Steam Achievement...   
1     Single-player, Steam Achievements, Steam Tradi...   
2     Single-player, Multi-player, Co-op, Steam Achi...   
3     Single-player, Full controller support, Steam ...   
4     Multi-player, Shared/Split Screen, Steam Achie...   
...                              

In [10]:
#BLOQUE METODO PARA RELLENAR CON VALOR 0 EL PRECIO DE LOS JUEGOS GRATUITOS

import pandas as pd

# Cargar el DataFrame desde el archivo CSV
df = pd.read_csv('C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/cambio_fechas_full_adri.csv')

# Rellenar los valores faltantes en la columna "Precio" con 0
df['Precio'] = df['Precio'].fillna(0)

# Guardar el DataFrame actualizado en un nuevo archivo CSV
print(df)
df.to_csv('C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/rellenar_price_adri.csv', index=False)


      ID del juego                      Géneros  \
0           330450        Action, Casual, Indie   
1           330680                Action, Indie   
2           275060            Action, Adventure   
3           331190                Action, Indie   
4           331220                Action, Indie   
...            ...                          ...   
4551        387400      Action, Indie, Strategy   
4552        400030  Casual, Free to Play, Indie   
4553        882110      Adventure, Free to Play   
4554       1265860               Racing, Sports   
4555       1089830   Racing, Simulation, Sports   

                                             Categorías  \
0     Single-player, Multi-player, Steam Achievement...   
1     Single-player, Steam Achievements, Steam Tradi...   
2     Single-player, Multi-player, Co-op, Steam Achi...   
3     Single-player, Full controller support, Steam ...   
4     Multi-player, Shared/Split Screen, Steam Achie...   
...                              

In [12]:
import pandas as pd
#BLOQUE METODO PARA LIMPIAR LOS REGISTROS LOS CUALES NO CUENTEN CON GENEROS & CATEGORIAS
# Leer el CSV
df = pd.read_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/full_dataframe_actual_adri.csv")

# Eliminar registros donde ambos Géneros y Categorías están vacíos
df = df.dropna(subset=['Géneros', 'Categorías'], how='all')

# Guardar el DataFrame modificado en un nuevo archivo CSV
df.to_csv("C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/clean_dataframe_actual_full_adri.csv", index=False)

print("Registros eliminados correctamente si cumplen con el criterio.")


Registros eliminados correctamente si cumplen con el criterio.


In [1]:
print("hello")

hello


In [3]:
#METODO PARA AGREGAR LOS MINUTOS JUGADOS DE CADA JUEGO, DEL DATAFRAME PRINCIPAL AL DATAFRAME DE GAMETAGS
import pandas as pd

# Cargar los datos desde los archivos CSV
dataA = pd.read_csv('C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/Principal/respuestas_A_modificado.csv')
dataB = pd.read_csv('C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/new_dataframe_actual_datesfull_roy.csv')

# Sumar los minutos jugados por ID de juego en dataA
minutos_por_juego = dataA.groupby('ID del juego')['Minutos jugadas'].sum().reset_index()

# Fusionar dataB con los minutos sumados por juego
dataB = pd.merge(dataB, minutos_por_juego, on='ID del juego', how='left')

# Renombrar la columna para reflejar los minutos jugados totales
dataB = dataB.rename(columns={'Minutos jugadas_x': 'Minutos jugadas', 'Minutos jugadas_y': 'Minutos jugadas Totales'})

# Guardar el resultado en un nuevo archivo CSV
dataB.to_csv('C:/Users/rodri/Documents/Python/Proyecto Integrador Big Data/data/TagsConseguidos/dataB_con_minutos_totales.csv', index=False)
